### Extract Data from Calls and Video Transcripts/Interviews

In [1]:
import os
from dotenv import load_dotenv

# For splitting transcript into pieces
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain

# Prompt templates for dynamic values
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate
)

# To create chat messages
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

load_dotenv()
openai_api_key = os.environ.get('OPENAI_API_KEY')

Now we load our data. It is an already formatted transcript of a mock sales call.

In [2]:
with open('./acme_co_v2.txt', 'r') as file:
    content = file.read()

In [3]:
print ("Transcript:\n")
print(content[:215])

Transcript:

Greg (Marin Transitions Partner): Does this say the call is recording for you at all? 

Eliza (ACME Co): But it says the request, the report or in here? 

Greg (Marin Transitions Partner): Nice. All right… one sec. 


Now we split our documents so that we don't run into token issues. 

In [4]:
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n"], chunk_size=2000, chunk_overlap=250)
texts = text_splitter.create_documents([content])

In [5]:
print (f"You have {len(texts)} texts")
texts[0]

You have 5 texts


Document(page_content="Greg (Marin Transitions Partner): Does this say the call is recording for you at all? \n\nEliza (ACME Co): But it says the request, the report or in here? \n\nGreg (Marin Transitions Partner): Nice. All right… one sec. \n\nGreg (Marin Transitions Partner): Cool. Absolutely. Well, I tell you what, thank you very much for joining me today. \n\nEliza (ACME Co): Yeah, glad to be chatting with you Greg. \n\nGreg (Marin Transitions Partner): Absolutely. Well. So, first of all, my name is Greg and I am from Marin transitions and we help people do social modeling on top of their businesses. So say somebody were to mention you on read it or mentioned you on Twitter or Facebook, then we help you understand what they're saying and the sentiment behind it. And I'm very excited to be chatting with you today. My first awesome. Well, I'll tell you what I've looked at your LinkedIn a little bit, but could you tell me more about your position and what you do? \n\nEliza (ACME Co):

In [6]:
llm = ChatOpenAI(temperature=0)

In [7]:
chain = load_summarize_chain(llm, chain_type='map_reduce', verbose=True)

In [9]:
output = chain.run(texts)



> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
Write a concise summary of the following:


"Greg (Marin Transitions Partner): Does this say the call is recording for you at all? 

Eliza (ACME Co): But it says the request, the report or in here? 

Greg (Marin Transitions Partner): Nice. All right… one sec. 

Greg (Marin Transitions Partner): Cool. Absolutely. Well, I tell you what, thank you very much for joining me today. 

Eliza (ACME Co): Yeah, glad to be chatting with you Greg. 

Greg (Marin Transitions Partner): Absolutely. Well. So, first of all, my name is Greg and I am from Marin transitions and we help people do social modeling on top of their businesses. So say somebody were to mention you on read it or mentioned you on Twitter or Facebook, then we help you understand what they're saying and the sentiment behind it. And I'm very excited to be chatting with you today. My first awesome. Well, I'll tell you what I've looked at your LinkedIn a littl

In [10]:
print(output)

Greg from Marin Transitions Partner and Eliza from ACME Co discuss ACME Co's social media activity and the need for a social monitoring tool. Eliza believes real-time understanding of customer sentiment could lead to closing more deals. ACME Co is looking for a tool that provides real-time data, scalability, and hands-on support. They are considering Hootsuite as an option. The implementation process is explained to Eliza, and she is satisfied. They discuss next steps for their partnership and personal interests before concluding the conversation.


Not bad, but it's giving us the perspective of a 3rd party watching the conversation that is agnostic to the content. I want the AI bot to me on our side! We'll need to switch up the prompts to do this.

### Custom Prompts

In [11]:
template="""

You are a helpful assistant that helps {sales_rep_name}, a sales rep at {sales_rep_company}, summarize information from a sales call.
Your goal is to write a summary from the perspective of {sales_rep_name} that will highlight key points that will be relevant to making a sale
Do not respond with anything outside of the call transcript. If you don't know, say, "I don't know"
Do not repeat {sales_rep_name}'s name in your output

"""
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = '{text}'
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [12]:
chat_prompt = ChatPromptTemplate.from_messages(messages=[system_message_prompt, human_message_prompt])

In [13]:
chain = load_summarize_chain(
    llm,
    chain_type='map_reduce',
    map_prompt=chat_prompt
)

In [14]:
output = chain.run({
    'input_documents': texts,
    'sales_rep_company': 'Marin Transitions Partner',
    'sales_rep_name': 'Greg'
})

In [15]:
print(output)

During a sales call, Greg from Marin Transitions Partner spoke with Eliza from ACME Co. Greg introduced his company and asked about Eliza's position and the size of ACME Co's marketing team. Eliza mentioned that ACME Co has 45 employees and a marketing team of seven. Greg also asked about ACME Co's technology stack and social media activity. Eliza expressed that ACME Co has no visibility into what customers are saying about them on social media and that this lack of transparency is a pain point for them. She estimated that having this transparency could help them close one more deal per month, worth around $50,000. Eliza expressed interest in a tool that provides social monitoring and visibility into their market. ACME Co is considering two vendors, including Marin Transitions Partner, and the evaluation process involves presenting recommendations to the CEO and conducting a legal and security review. Eliza mentioned that they are also considering Hootsuite. Greg thanked Eliza for her 

Better! But say I wanted to change the format of the output without needing the user to do extra prompting.

### Promptless Changes

In [16]:
summary_output_options = {
    'one_sentence' : """
     - Only one sentence
    """,
    
    'bullet_points': """
     - Bullet point format
     - Separate each bullet point with a new line
     - Each bullet point should be concise
    """,
    
    'short' : """
     - A few short sentences
     - Do not go longer than 4-5 sentences
    """,
    
    'long' : """
     - A verbose summary
     - You may do a few paragraphs to describe the transcript if needed
    """
}

Create a new template that takes an additional parameter. I need to put this in the combined prompt so that the LLM will output in my format. If I did this in the map section I would lose the format after the combined prompt was done

**Map Prompt**

In [17]:
template="""

You are a helpful assistant that helps {sales_rep_name}, a sales rep at {sales_rep_company}, summarize information from a sales call.
Your goal is to write a summary from the perspective of Greg that will highlight key points that will be relevant to making a sale
Do not respond with anything outside of the call transcript. If you don't know, say, "I don't know"
"""
system_message_prompt_map = SystemMessagePromptTemplate.from_template(template)

human_template="{text}" # Simply just pass the text as a human message
human_message_prompt_map = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt_map = ChatPromptTemplate.from_messages(messages=[system_message_prompt_map, human_message_prompt_map])

**Combined Prompt**

In [18]:
template="""

You are a helpful assistant that helps {sales_rep_name}, a sales rep at {sales_rep_company}, summarize information from a sales call.
Your goal is to write a summary from the perspective of Greg that will highlight key points that will be relevant to making a sale
Do not respond with anything outside of the call transcript. If you don't know, say, "I don't know"

Respond with the following format
{output_format}

"""
system_message_prompt_combine = SystemMessagePromptTemplate.from_template(template)

human_template="{text}" # Simply just pass the text as a human message
human_message_prompt_combine = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt_combine = ChatPromptTemplate.from_messages(messages=[system_message_prompt_combine, human_message_prompt_combine])

In [19]:
chain = load_summarize_chain(llm,
                             chain_type="map_reduce",
                             map_prompt=chat_prompt_map,
                             combine_prompt=chat_prompt_combine,
                             verbose=True
                            )

In [20]:
user_selection = 'one_sentence'

output = chain.run({
                    "input_documents": texts,
                    "sales_rep_company": "Marin Transitions Partner", \
                    "sales_rep_name" : "Greg",
                    "output_format" : summary_output_options[user_selection]
                   })



> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
System: 

You are a helpful assistant that helps Greg, a sales rep at Marin Transitions Partner, summarize information from a sales call.
Your goal is to write a summary from the perspective of Greg that will highlight key points that will be relevant to making a sale
Do not respond with anything outside of the call transcript. If you don't know, say, "I don't know"

Human: Greg (Marin Transitions Partner): Does this say the call is recording for you at all? 

Eliza (ACME Co): But it says the request, the report or in here? 

Greg (Marin Transitions Partner): Nice. All right… one sec. 

Greg (Marin Transitions Partner): Cool. Absolutely. Well, I tell you what, thank you very much for joining me today. 

Eliza (ACME Co): Yeah, glad to be chatting with you Greg. 

Greg (Marin Transitions Partner): Absolutely. Well. So, first of all, my name is Greg and I am from Marin transitions and we help people do social m

In [21]:
print(output)

Summary:

During the sales call, Greg from Marin Transitions Partner discussed the next steps with Eliza from ACME Co, including sharing a brochure and pricing information, and scheduling a follow-up meeting for next week. They also briefly discussed personal interests, with Eliza mentioning her Australian Shepherd dog and hobbies like hiking and wine tasting.
